In [73]:
import os
from pathlib import Path
from cryptography.fernet import Fernet

In [ ]:
home = str(Path.home())

In [74]:
file = open(home+'/sql_files/pysqlkey.key', 'rb')
key = file.read() # The key will be type bytes
file.close()

cipher_suite = Fernet(key)
with open(home+'/sql_files/sql_server_pwd.bin', 'rb') as file_object:
    for line in file_object:
        encryptedpwd = line

import pyodbc
server = 'deep-learning'
database = 'MScDB'
username = 'sa'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ bytes((cipher_suite.decrypt(encryptedpwd))).decode("utf-8"))
cursor = cnxn.cursor()

In [75]:
weather_path = '/media/alessio/3TB/Dropbox/pi/weather_data/'

In [76]:
os.chdir(weather_path)

In [77]:
print(os.getcwd())

/media/alessio/3TB/Dropbox/pi/weather_data


In [78]:
weather_file_name = 'weather_data_files.txt'
weather_files = open(weather_file_name, "r",encoding='utf-8')

In [79]:
weather_file_filenames = weather_files.readlines()

In [80]:
weather_files.close()

In [81]:
len(weather_file_filenames)

9251

In [82]:
def get_weather_data (weather_file_Content):
    conditions = weather_file_Content[1].strip('\n')
    temperature = weather_file_Content[2].strip('\n')
    wind = weather_file_Content[3].strip('\n')
    UVIndex = weather_file_Content[5].strip('\n')
    rain = weather_file_Content[7].strip('\n')
    heatStress = weather_file_Content[9].strip('\n')
    sunrise = weather_file_Content[11].strip('\n')
    feelsLike = weather_file_Content[13].strip('\n')
    sunset = weather_file_Content[15].strip('\n')
    gustBft = weather_file_Content[17].strip('\n')
    seaTemp = weather_file_Content[19].strip('\n')
    humidity = weather_file_Content[21].strip('\n')
    pressure = weather_file_Content[23].strip('\n')
    weather_line = {
        "filename"     : weather_file.name
        ,"conditions"   : conditions 
        ,"temperature" : temperature
        ,"wind"        : wind 
        ,"UVIndex"     : UVIndex  
        ,"rain"        : rain
        ,"heatStress"  : heatStress   
        ,"sunrise"     : sunrise  
        ,"feelsLike"   : feelsLike 
        ,"sunset"      : sunset 
        ,"gustBft"     : gustBft 
        ,"seaTemp"     : seaTemp 
        ,"humidity"    : humidity  
        ,"pressure"    : pressure 
    }
    return weather_line

In [83]:
def insert_into_db (weather_line_data):
    #Insert Query
    tsql = "INSERT INTO weather_data\
    (filename,conditions,temperature,wind,UVIndex,rain,heatStress,sunrise,feelsLike,sunset,gustBft,seaTemp,humidity,pressure)\
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?);"
    cursor.execute(tsql
        ,weather_line_data.get('filename')
        ,weather_line_data.get('conditions')
        ,weather_line_data.get('temperature')
        ,weather_line_data.get('wind')
        ,weather_line_data.get('UVIndex')
        ,weather_line_data.get('rain')
        ,weather_line_data.get('heatStress')
        ,weather_line_data.get('sunrise')
        ,weather_line_data.get('feelsLike')
        ,weather_line_data.get('sunset')
        ,weather_line_data.get('gustBft')
        ,weather_line_data.get('seaTemp')
        ,weather_line_data.get('humidity')
        ,weather_line_data.get('pressure'))

In [84]:
for x in range(0,len(weather_file_filenames)):
    weather_file = open(weather_file_content[x].strip('\n'))
    weather_file_data = weather_file.readlines()
    weather_line_dict = get_weather_data(weather_file_data)
    insert_into_db(weather_line_dict)
    weather_file.close()

In [85]:
cursor.commit()
cursor.close()
cnxn.close()